In [ ]:
import os
import sys
from pathlib import Path
# Setting the working directory to the root of the project
project_dir = Path("C:/Users/adbou/source/repos/KFHXRelatedAi/")
os.chdir(project_dir)

from Configs.GeneralPaths import SOURCEDATA
import pandas as pd
import numpy as np
import ast


In [ ]:
#load user transactions data
deals_embeddings = pd.read_csv(Path(SOURCEDATA / "Deals_Embeddings.csv"))
deals_embeddings['ada_embedding'] = deals_embeddings['ada_embedding'].apply(ast.literal_eval)



In [ ]:
deals_embeddings.head()

In [ ]:
#loading user transactions
user_transactions = pd.read_excel(Path(SOURCEDATA / "Transaction_User.xlsx"))
user_tranactions = user_transactions.drop(columns=['TrxId'])
user_tranactions.head()

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def calculate_cosine_similarity(embedding, embeddings):
    similarities = cosine_similarity([embedding], embeddings)
    return similarities[0]


In [ ]:
def recommend_deals_for_user(user_id, user_deals_df, deals_embeddings_new):
    # Get the user's redeemed deals' content IDs
    redeemed_content_ids = user_deals_df[user_deals_df['FK_BusinessUserId'] == user_id]['FK_ContentId'].unique()
    
    # Retrieve the embeddings for the redeemed deals
    redeemed_embeddings = deals_embeddings_new[deals_embeddings_new['ContentId'].isin(redeemed_content_ids)]
    
    if redeemed_embeddings.empty:
        print("No redeemed deals found for this user.")
        return pd.DataFrame()
    
    # Get all embeddings
    all_embeddings = deals_embeddings_new['ada_embedding'].tolist()

    # Initialize an empty list to store similarities
    similarities = []

    # Calculate similarities for each redeemed embedding
    for embedding in redeemed_embeddings['ada_embedding']:
        similarity = calculate_cosine_similarity(embedding, all_embeddings)
        similarities.append(similarity)

    # Average the similarities to get a single similarity score for each deal
    average_similarities = np.mean(similarities, axis=0)

    # Add the similarities to the DataFrame
    deals_embeddings_new['similarity'] = average_similarities

    # Filter out deals the user has already redeemed
    recommended_deals = deals_embeddings_new[~deals_embeddings_new['ContentId'].isin(redeemed_content_ids)]

    # Sort by similarity in descending order and take the top 5 recommendations
    top_5_recommendations = recommended_deals.sort_values(by='similarity', ascending=False).head(5)

    return top_5_recommendations




In [ ]:
user_id = 1002107

# Get recommendations for the user
top_5_recommendations = recommend_deals_for_user(user_id, user_tranactions, deals_embeddings)

In [ ]:
top_5_recommendations

In [ ]:
user_trx = user_tranactions[user_tranactions['FK_BusinessUserId'] == 1070553]
user_trx